In [ ]:
%load_ext aiida
%aiida
import urllib.parse as urlparse

import ipywidgets as ipw
from aiida import orm
from aiidalab_widgets_base import viewer

from surfaces_tools import helpers
from surfaces_tools.widgets import comments, obsolete

In [ ]:
pk = urlparse.parse_qs(urlparse.urlsplit(jupyter_notebook_url).query)["pk"][0]
workcalc = orm.load_node(pk)
opt_structure = workcalc.outputs.output_structure

# Equilibrium geometry

In [ ]:
display(viewer(opt_structure))

# Output parameters

In [ ]:
info = ipw.HTML()
display(info)
energy = workcalc.outputs.output_parameters.get_dict()["energy"]
energy_scf = workcalc.outputs.output_parameters.get_dict()["energy_scf"]
try:
    dispersion_energy_au = workcalc.outputs.output_parameters.get_dict()[
        "motion_step_info"
    ]["dispersion_energy_au"][-1]
except IndexError:
    dispersion_energy_au = None
dft_type = workcalc.outputs.output_parameters.get_dict()["dft_type"]
run_type = workcalc.outputs.output_parameters.get_dict()["run_type"]

html = f"<b>Equilibrim geometry PK-{opt_structure.pk}</b> <br>"
html += "<b>Energy:</b> {} (Hartree), {} (eV)<br>".format(
    energy, round((energy * helpers.HART_2_EV), 3)
)
html += "<b>Energy scf:</b> {} (Hartree), {} (eV) <br>".format(
    energy_scf, round((energy_scf * helpers.HART_2_EV), 3)
)
if dispersion_energy_au is not None:
    html += "<b>Dispersion Energy:</b> {} (Hartree), {} (eV)<br>".format(
        dispersion_energy_au, round((dispersion_energy_au * helpers.HART_2_EV), 3)
    )
# patch for wrong gap in the parser
gap = []
with workcalc.outputs.retrieved.open("aiida.out") as fhandle:
    lines = fhandle.readlines()
for line in lines:
    if line.startswith(" HOMO - LUMO gap"):
        gap.append(float(line.split()[-1]))
if dft_type == "UKS":
    html += "<b>Gap spin1: </b> %s (eV)<br>" % round(gap[0], 3)
    html += "<b>Gap spin2: </b> %s (eV)<br>" % round(gap[1], 3)
else:
    html += "<b>Gap: </b> %s (eV)<br>" % round(gap[0], 3)
# end patch
if run_type == "CELL_OPT":
    html += (
        "<b>Cell a (A): </b> %s <br>"
        % workcalc.outputs.output_parameters.get_dict()["motion_step_info"][
            "cell_a_angs"
        ][-1]
    )
    html += (
        "<b>Cell b (A): </b> %s <br>"
        % workcalc.outputs.output_parameters.get_dict()["motion_step_info"][
            "cell_b_angs"
        ][-1]
    )
    html += (
        "<b>Cell c (A): </b> %s <br>"
        % workcalc.outputs.output_parameters.get_dict()["motion_step_info"][
            "cell_c_angs"
        ][-1]
    )
    html += (
        "<b>alpha (DEG): </b> %s <br>"
        % workcalc.outputs.output_parameters.get_dict()["motion_step_info"][
            "cell_alp_deg"
        ][-1]
    )
    html += (
        "<b>beta (DEG): </b> %s <br>"
        % workcalc.outputs.output_parameters.get_dict()["motion_step_info"][
            "cell_bet_deg"
        ][-1]
    )
    html += (
        "<b>gamma (DEG): </b> %s <br>"
        % workcalc.outputs.output_parameters.get_dict()["motion_step_info"][
            "cell_gam_deg"
        ][-1]
    )
    html += (
        "<b>Pressure (bar): </b> %s <br>"
        % workcalc.outputs.output_parameters.get_dict()["motion_step_info"][
            "pressure_bar"
        ][-1]
    )

info.value = html

# Comments

In [ ]:
comments_widget = comments.CommentsWidget(workchain=pk)
display(comments_widget)

# Mark calculation as obsolete 

In [ ]:
obsolete = obsolete.ObsoleteWidget(workchain=pk)
display(obsolete)